In [1]:
#!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

In [2]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

In [3]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset

import dgl
import dgl.function as fn

import networkx as nx
from matplotlib import pyplot as plt
import numpy as np

import os
import gc

# デバイス設定
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


In [17]:
from scipy import special
from torch.autograd import Function

In [18]:
def printNPZ(npz):
    for kw in npz.files:
        print(kw, npz[kw])

In [19]:
dirName = '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/'
savedirName = dirName + 'HiraiwaModelFit_multiStep/'
os.makedirs(savedirName, exist_ok=True)

params = np.load(dirName+'params.npz')
#traj = np.load(dirName+'result.npz')

printNPZ(params)
#printNPZ(traj)

v0 1.0
r 1.0
D 0.1
A 0.0
L 20
rho 1.0
beta 1.0
A_CFs [0.1 0.9]
A_CIL 0.0
cellType_ratio [0.5 0.5]
quiv_colors ['k' 'r']
kappa 0.5
A_Macdonalds [2.  0.2]
batch_size 400
state_size 3
brownian_size 1
periodic True
t_max 1000
methodSDE heun
isIto False
stepSDE 0.01


In [20]:
if params['periodic']:
    L = torch.tensor(params['L'])
    def calc_dr(r1, r2):
        dr = torch.remainder((r1 - r2), L)
        dr[dr > L/2] = dr[dr > L/2] - L
        return dr
else:
    def calc_dr(r1, r2):
        return r1 - r2
    
def makeGraph(x_data, r_thresh):
        Ndata = x_data.size(0)
        dx = calc_dr(torch.unsqueeze(x_data, 0), torch.unsqueeze(x_data, 1))
        dx = torch.sum(dx**2, dim=2)
        edges = torch.argwhere(dx < r_thresh/2)
        return dgl.graph((edges[:,0], edges[:,1]), num_nodes=Ndata)

In [21]:
subdir_list = [f.path for f in os.scandir(dirName) if f.is_dir()]

print(subdir_list)

datadir_list = [f for f in subdir_list if 'result.npz' in [ff.name for ff in os.scandir(f)]]

print(datadir_list)

['/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221013_130509', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221013_222508', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221014_020927', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221014_055334', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221014_093729', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/ActiveNet_celltypes', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/ActiveNet_vp_rotsym', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221022_000642', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/ActiveNet_vp_rotsym_dropout', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/ActiveNet_vp_rotsym_batchNorm', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221024_041213', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221024_080435', '/home/uwamichi/jupyter/HiraiwaModel_chem20220916_150816/20221024

In [22]:
class myDataset(Dataset):
    def __init__(self, data_x, celltype_List, t_yseq=1):
        super().__init__()
        
        self.data_x = data_x # List of tensors
        #self.data_y = data_y
        self.celltype_List = celltype_List
        
        self.data_len = np.array([xx.size(0) for xx in self.data_x])
        self.t_yseq = t_yseq
        
        self.data_len_cumsum = np.cumsum(self.data_len - (self.t_yseq - 1))
        
    def __len__(self):
        return (self.data_len - (self.t_yseq - 1)).sum()
    
    def __getitem__(self, index):
        id_List = np.argwhere(index<self.data_len_cumsum)[0,0]
        
        if id_List:
            id_tensor = index - self.data_len_cumsum[id_List-1]
        else:
            id_tensor = index
        
        return self.data_x[id_List][id_tensor], self.data_x[id_List][id_tensor:(id_tensor+self.t_yseq)], self.celltype_List[id_List]

In [23]:
dr_thresh = 7
dt = 1
batch_size = 8

T_pred = 3

N_data = len(datadir_list)

#TR_VA_rate = np.array([0.6, 0.2])

TR_last = 5
VA_last = 7

shuffle_inds = np.arange(N_data, dtype=int)
np.random.shuffle(shuffle_inds)

train_inds = shuffle_inds[:TR_last]
valid_inds = shuffle_inds[TR_last:VA_last]
test_inds = shuffle_inds[VA_last:]

celltype_lst = []

train_x = []
valid_x = []
test_x = []

train_y = []
valid_y = []
test_y = []

train_ct = []
valid_ct = []
test_ct = []

for i_dir, subdirName in enumerate(datadir_list):
    
    traj = np.load(subdirName+'/result.npz')
    
    xy_t = torch.tensor(traj['xy'])#[:-1,:,:])
    #v_t = calc_dr(torch.tensor(traj['xy'][1:,:,:]), torch.tensor(traj['xy'][:-1,:,:])) / dt
    p_t = torch.unsqueeze(torch.tensor(traj['theta']), dim=2)#[:-1,:]), dim=2)
    #w_t = torch.unsqueeze(torch.tensor((traj['theta'][1:,:]-traj['theta'][:-1,:])%(2*np.pi)/dt), dim=2)
    
    if i_dir in train_inds:
        train_x.append(torch.concat((xy_t, p_t), -1))
        #train_y.append(torch.concat((v_t, w_t), -1))
        train_ct.append(torch.tensor(traj['celltype_label']).view(-1,1))

    if i_dir in valid_inds:
        valid_x.append(torch.concat((xy_t, p_t), -1))
        #valid_y.append(torch.concat((v_t, w_t), -1))
        valid_ct.append(torch.tensor(traj['celltype_label']).view(-1,1))
        
    if i_dir in test_inds:
        test_x.append(torch.concat((xy_t, p_t), -1))
        #test_y.append(torch.concat((v_t, w_t), -1))
        test_ct.append(torch.tensor(traj['celltype_label']).view(-1,1))
    
train_dataset = myDataset(train_x, train_ct, t_yseq=T_pred)

valid_dataset = myDataset(valid_x, valid_ct, t_yseq=T_pred)

test_dataset = myDataset(test_x, test_ct, t_yseq=T_pred)


train_data = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory=True)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

del train_x, train_ct, train_dataset
del valid_x, valid_ct, valid_dataset
del test_x, test_ct, test_dataset
gc.collect()

#print(data)
#print(data.num_graphs)
#print(data.x)
#print(data.y)
#print(data.edge_index)

366

In [24]:
print(train_inds)

[ 1  8  0 10  9]


In [25]:
dir(train_data)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_get_shared_seed',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_i

In [26]:
len(train_data)

625

In [27]:
print(train_data.dataset.data_len_cumsum)

[ 999 1998 2997 3996 4995]


In [82]:
class torch_knFunction(Function):
    @staticmethod
    def forward(ctx, input, n):
        ctx.save_for_backward(input)
        ctx.n = n
        numpy_input = input.cpu().detach().numpy()
        result = special.kn(n, numpy_input)
        return torch.as_tensor(result, dtype=input.dtype, device=device)
    
    @staticmethod
    def backward(ctx, grad_output):
        numpy_go = grad_output.cpu().detach().numpy()
        input, = ctx.saved_tensors
        n = ctx.n
        numpy_input = input.cpu().detach().numpy()
        if n==0:
            grad_kn = -special.kn(1, numpy_input)
        else:
            grad_kn = -(special.kn(n-1, numpy_input) + special.kn(n+1, numpy_input))/2
        result = numpy_go * grad_kn
        return torch.as_tensor(result, dtype=input.dtype, device=device), None

class torch_kn(nn.Module):
    def __init__(self, n):
        super(torch_kn, self).__init__()
        self.n = n
        
    def forward(self, input):
        return torch_knFunction.apply(input, self.n)
    
torch_scipy_k0 = torch_kn(0)
torch_scipy_k1 = torch_kn(1)

cutoff = torch.tensor([3.5], device = device)
k1_cutoff = torch_scipy_k1(cutoff)

func_cutoff = nn.ReLU()

def torch_scipy_k1_cutoff(x):
    return func_cutoff(torch_scipy_k1(x) - k1_cutoff)

In [83]:
def J_chemMacdonald(xy, d): # cutoff! --> shift the values
    chemMag = torch_scipy_k1_cutoff(params['kappa'].item() * d) *(params['kappa'].item()/(2*np.pi))
    xy = chemMag * xy
    return torch.nansum(xy, dim=0, keepdim=False)    

In [95]:
if params['periodic']:
    def xy2distance(xy, L):
        dx = xy[:, :1] - xy[:, :1].T   # neighbor - target
        dy = xy[:, 1:] - xy[:, 1:].T
        dx = torch.remainder(dx, L)
        dy = torch.remainder(dy, L)
        dx[dx > L/2] = dx[dx > L/2] - L
        dy[dy > L/2] = dy[dy > L/2] - L
        return [dx, dy]
else:
    def xy2distance(xy, L):
        dx = xy[:, :1] - xy[:, :1].T
        dy = xy[:, 1:] - xy[:, 1:].T
        return [dx, dy]    

def J_CF(xy, dr, q):
    c = torch.unsqueeze(q[0], 1)   # cos, sin of neighbor
    s = torch.unsqueeze(q[1], 1)
    xy = (1 + xy[:,:,:1] * c + xy[:,:,1:] * s) * xy / 2
    return torch.nansum(xy, dim=0, keepdim=False)

def J_CIL(xy, d, r):
    xy = ((r/d) - 1) * xy
    xy[...,:1][d==0] = 0
    xy[...,1:][d==0] = 0
    return torch.nansum(xy, dim=0, keepdim=False)

In [96]:
class NeuralNet(nn.Module):
    def __init__(self, L, periodic, v0, beta, A_CF, A_CIL, r, A, A_Macdonald):
        super().__init__()
        self.L = torch.tensor(L, device=device)
        self.periodic = periodic
        self.v0 = nn.Parameter(torch.tensor([[v0]], requires_grad=True, device=device))
        self.beta = nn.Parameter(torch.tensor([[beta]], requires_grad=True, device=device))
        self.A_CF = nn.Parameter(torch.unsqueeze(torch.tensor(A_CF, requires_grad=True, device=device), 1))
        self.A_CIL = A_CIL
        self.r = torch.tensor(r, device=device)
        self.A = A
        self.A_Macdonald = nn.Parameter(torch.unsqueeze(torch.tensor(A_Macdonald, requires_grad=True, device=device), 1))
        
    def forward(self, x, ct):
        xy = x[:, :2]
#        if self.periodic:
#            xy = xy % L
        xy = xy2distance(xy, self.L)
#        print(np.shape(xy))
#        print(xy[0].shape)
        xy = torch.cat((torch.unsqueeze(xy[0], 2), torch.unsqueeze(xy[1], 2)), 2)
        d = torch.norm(xy, p='fro', dim=2, keepdim=True)   # distance
        dr = 0.5 + (torch.atan((self.r - d) * 100) / (np.pi))   # 1 if distance < r, else 0
        xy = dr * torch.nn.functional.normalize(xy, p=2.0, dim=2)   # normalized distance vector
        c = torch.cos(x[:, 2:])
        s = torch.sin(x[:, 2:])

        jcil = J_CIL(xy, d, self.r)
        
        jchem = J_chemMacdonald(xy, d)
        
        dx0 = self.v0 * torch.cat((c, s), 1)
        dx1 = -self.beta * jcil
        dtheta0 = torch.index_select(self.A_CF, 0, ct[:,0]) * J_CF(xy, dr, [c,s]) - self.A_CIL * jcil
        dtheta0 = c * dtheta0[:, 1:] - s * dtheta0[:, :1]
        dtheta1 = self.A * c
        dtheta2 = torch.index_select(self.A_Macdonald, 0, ct[:,0]) * (c * jchem[:, 1:] - s * jchem[:, :1])
        
        return torch.cat((x[:,:2]+dx0+dx1, x[:,2:]+dtheta0+dtheta1+dtheta2), 1)  # shape (batch_size, state_size)



In [97]:
def myLoss(out, target):
    #dv = torch.sum(torch.square(out[..., :xy_dim] - target[..., :xy_dim]), dim=-1)
    dv = torch.sum(torch.square(calc_dr(out[..., :xy_dim], target[..., :xy_dim])), dim=-1)
    dcos = torch.cos(out[..., xy_dim] - target[..., xy_dim])
    
    wei_shape = np.ones([dv.dim()], dtype=int)
    wei_shape[0] = T_pred
    wei = torch.tensor(np.reshape(1/np.arange(1, T_pred+1), wei_shape)).to(dv.device)
    wei = wei/wei.mean()
    
    return torch.mean(dv*wei), torch.mean((1-dcos)*wei)

In [98]:
# モデルのインスタンス生成
xy_dim = 2

model = NeuralNet(params['L'],
                  params['periodic'], 
                  0., 0., [0.,0.], 0., params['r'], 0., [0.,0.]).to(device)
# input data
#data = dataset[0]

def calc_multiSteps(x0, ct):
    outs = []
    x_i = x0
    for i_step in range(T_pred):
        x_i = x_i + model(x_i, ct.int()) * dt
        outs.append(x_i.clone())
    return torch.stack(outs, dim=0)

# optimizer
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-7)#, weight_decay=5e-4)
#optimizer = torch.optim.Adadelta(model.parameters())#, rho=0.95)#, lr=1e-1, momentum=0.9)

scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

val_loss_log = []

val_loss_min = np.Inf

# learnig loop
for epoch in range(50):
    model.train()
    for batch_x, batch_y, batch_ct in train_data:
        optimizer.zero_grad()
        lossv = 0
        losstheta = 0
        for ib in range(batch_x.size(0)):
            out = calc_multiSteps(batch_x[ib].to(device), batch_ct[ib].to(device))
            lv, ltheta = myLoss(out, batch_y[ib].to(device))
            lossv = lossv + lv
            losstheta = losstheta + ltheta
        lossv = lossv / batch_x.size(0)
        losstheta = losstheta / batch_x.size(0)
        (lossv+losstheta).backward()
        optimizer.step()
    model.eval()
    val_lossv = 0
    val_losstheta = 0
    val_count = 0
    with torch.no_grad():
        for batch_x, batch_y, batch_ct in valid_data:
            for ib in range(batch_x.size(0)):
                val_out = calc_multiSteps(batch_x[ib].to(device), batch_ct[ib].to(device))
                lv, ltheta = myLoss(val_out, batch_y[ib].to(device))
                val_lossv = val_lossv + lv
                val_losstheta = val_losstheta + ltheta
            val_count = val_count + batch_x.size(0)
    val_lossv = val_lossv/val_count
    val_losstheta = val_losstheta/val_count
    val_loss = val_lossv + val_losstheta
    scheduler.step(val_loss)
    print('Epoch %d | train Loss: [%.4f, %.4f] | valid Loss: [%.4f, %.4f]' % (epoch,
                                                                              lossv.item(), 
                                                                              losstheta.item(),
                                                                              val_lossv.item(), 
                                                                              val_losstheta.item()))
    val_loss_log.append([val_lossv.cpu().item(), val_losstheta.cpu().item()])
    if val_loss.item() < val_loss_min:
        stored_model = model
        val_loss_min = val_loss.item()

/tmp/ipykernel_18371/2533986396.py:20: RuntimeWarning: invalid value encountered in multiply
  result = numpy_go * grad_kn


Epoch 0 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 1 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 2 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 3 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 4 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 00006: reducing learning rate of group 0 to 1.5000e-07.
Epoch 5 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 6 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 7 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 8 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 9 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 10 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 00012: reducing learning rate of group 0 to 7.5000e-08.
Epoch 11 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 12 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 13 | train Loss: [nan, nan] | valid Loss: [nan, nan]
Epoch 14 | train Loss: [nan, nan] | valid Loss: [nan, nan]
E

In [99]:
print(batch_x.shape)
print(batch_y.shape)
print(batch_ct.shape)

torch.Size([6, 400, 3])
torch.Size([6, 3, 400, 3])
torch.Size([6, 400, 1])


In [100]:
model.state_dict()

OrderedDict([('v0', tensor([[nan]], device='cuda:1')),
             ('beta', tensor([[nan]], device='cuda:1')),
             ('A_CF',
              tensor([[nan],
                      [nan]], device='cuda:1')),
             ('A_Macdonald',
              tensor([[nan],
                      [nan]], device='cuda:1'))])

In [101]:
# モデルを評価モードに設定
stored_model.eval()

# 推論
test_lossv = 0
test_losstheta = 0
test_count = 0
with torch.no_grad():
    for batch_x, batch_y, batch_ct in test_data:
        for ib in range(batch_x.size(0)):
            test_out = calc_multiSteps(batch_x[ib].to(device), batch_ct[ib].to(device))
            lv, ltheta = myLoss(test_out, batch_y[ib].to(device))
            test_lossv = test_lossv + lv
            test_losstheta = test_losstheta + ltheta
        test_count = test_count + batch_x.size(0)
test_lossv = test_lossv/test_count
test_losstheta = test_losstheta/test_count
print('test Loss: [%.4f, %.4f]' % (test_lossv.item(), test_losstheta.item()))
test_loss = [test_lossv.item(), test_losstheta.item()]

NameError: name 'stored_model' is not defined

In [ ]:
import pickle
import datetime

now = datetime.datetime.now()
nowstr = now.strftime('%Y%m%d_%H%M%S')

os.makedirs(savedirName + nowstr + '/', exist_ok=True)

In [ ]:
stored_model.selfpropel.detach()

In [ ]:
stored_model = stored_model.to('cpu')

filename1 = savedirName + nowstr + '/' + nowstr + '_Model.pkl'
with open(filename1, "wb") as f:
    pickle.dump(stored_model, f)

filename1_2 = savedirName + nowstr + '/' + nowstr + '_Model.pt'
torch.save(stored_model, filename1_2)

filename2 = savedirName + nowstr + '/' + nowstr
torch.save(stored_model.interactNN.state_dict(), filename2 + '_interactNN.pkl')
torch.save(stored_model.thetaDotNN.state_dict(), filename2 + '_thetaDotNN.pkl')
torch.save(stored_model.selfpropel.detach(), filename2 + '_selfpropel.pkl')

filename3 = savedirName + nowstr + '/' + nowstr + '_Separation.npz'
np.savez(filename3, dr_thresh=dr_thresh, T_pred=T_pred, batch_size=batch_size,
         train_inds=train_inds, valid_inds=valid_inds, test_inds=test_inds, 
         val_loss_log=val_loss_log, test_loss=test_loss)

filename4 = savedirName + nowstr + '/' + nowstr + '_optimizer.pt'
torch.save(optimizer, filename4)